In [1]:
import src.config as model_config
from src.utils import get_project_root_path
import src.data
import src.metrics
import src.plot

from src.model import (
    ProstT5EncoderModelForSequenceClassification,
    ProstT5EncoderModelTokenClassificationCRF,
    ProstT5EncoderModelTokenClassificationLinear,
    ProtT5EncoderModelForSequenceClassification,
    ProtT5EncoderModelTokenClassificationCRF,
    ProtT5EncoderModelTokenClassificationLinear,
)

import gc
import random
import tqdm

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from transformers import (
    T5Tokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    TrainerCallback,
    set_seed
)

import peft
from peft import (
    LoraConfig,
)

In [2]:
ROOT = get_project_root_path()
device = torch.device('cuda:0' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))

EXPERT = model_config.selected_expert
MODEL_VERRSION = model_config.model_version

adapter_location = f'/models/moe_v{MODEL_VERRSION}_CRF_expert_{EXPERT}'

SEED = model_config.seed
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
set_seed(SEED)

print("Base Model:\t", model_config.base_model_name)
print("MPS Availible:\t", torch.backends.mps.is_available())
print("Path:\t\t", ROOT)
print(f"Using device:\t {device}")

Base Model:	 Rostlab/prot_t5_xl_uniref50
MPS Availible:	 True
Path:		 /Users/finnlueth/Developer/gits/SignalGPT
Using device:	 mps


In [3]:
t5_tokenizer = T5Tokenizer.from_pretrained(
    pretrained_model_name_or_path=model_config.base_model_name,
    do_lower_case=False,
    use_fast=True,
    legacy=False,
)